# Глубинное обучение для текстовых данных, ФКН ВШЭ
## Домашнее задание 4: Уменьшение размеров модели
### Оценивание и штрафы

Максимально допустимая оценка за работу — __10 баллов__.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов. Весь код должен быть написан самостоятельно. Чужим кодом для пользоваться запрещается даже с указанием ссылки на источник. В разумных рамках, конечно. Взять пару очевидных строчек кода для реализации какого-то небольшого функционала можно.

Неэффективная реализация кода может негативно отразиться на оценке. Также оценка может быть снижена за плохо читаемый код и плохо оформленные графики. Все ответы должны сопровождаться кодом или комментариями о том, как они были получены.

__Мягкий дедлайн 29.11.24 23:59__ \
__Жесткий дедлайн 2.12.24 23:59__

### О задании

В этом задании вам предстоит научиться решать задачу Named Entity Recognition (NER) на самом популярном датасете – [CoNLL-2003](https://paperswithcode.com/dataset/conll-2003). В вашем распоряжении будет предобученный BERT, который вам необходимо уменьшить с минимальными потерями в качестве до размера 20М параметров. Для этого вы самостоятельно реализуете факторизацию эмбеддингов, дистилляцию, шеринг параметров и так далее.

В этом задании вам придется проводить довольно много экспериментов, поэтому мы рекомендуем не писать весь код в тетрадке, а завести разные файлы для отдельных логических блоков и скомпоновать все в виде проекта. Это позволит вашему ноутбуку не разрастаться и сильно облегчит задачу и вам, и проверяющим. Так же постарайтесь логгировать все ваши эксперименты в wandb, чтобы ничего не потерялось.

### Оценивание
Оценка за это домашнее задание будет формироваться из оценки за __задания__ и за __отчет__, в котором, как и раньше, от вас требуется написать о проделанной работе. За отчет можно получить до 2-х баллов, однако в случае отсутствия отчета можно потерять баллы за сами задания. Задания делятся на две части: _номерные_ и _на выбор_. За _номерные_ можно получить в сумме 6 баллов, за задания _на выбор_ можно получить до 16. То есть за все дз можно получить 24 балла. Все, что вы наберете свыше 10, будет считаться бонусами.


### О датасете

Named Entity Recognition – это задача классификации токенов по классам сущностей. В CoNLL-2003 для именования сущностей используется маркировка **BIO** (Beggining, Inside, Outside), в которой метки означают следующее:

- *B-{метка}* – начало сущности *{метка}*
- *I-{метка}* – продолжнение сущности *{метка}*
- *O* – не сущность

Существуют так же и другие способы маркировки, например, BILUO. Почитать о них можно [тут](https://en.wikipedia.org/wiki/Inside–outside–beginning_(tagging)) и [тут](https://www.youtube.com/watch?v=dQw4w9WgXcQ).

Всего в датасете есть 9 разных меток.
- O – слову не соответствует ни одна сущность.
- B-PER/I-PER – слово или набор слов соответстует определенному _человеку_.
- B-ORG/I-ORG – слово или набор слов соответстует определенной _организации_.
- B-LOC/I-LOC – слово или набор слов соответстует определенной _локации_.
- B-MISC/I-MISC – слово или набор слов соответстует сущности, которая не относится ни к одной из предыдущих. Например, национальность, произведение искусства, мероприятие и т.д.

Приступим!

Начнем с загрузки и предобработки датасета.

In [1]:
import torch
import numpy as np
from datasets import DatasetDict
from seqeval.metrics import f1_score, classification_report

from datasets import DatasetDict
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments, DataCollatorForTokenClassification
from transformers.tokenization_utils import BatchEncoding

In [2]:
dataset = load_dataset("eriktks/conll2003")

dataset = dataset.remove_columns(["id", "pos_tags", "chunk_tags"])
label_names = ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']
dataset

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 3453
    })
})

In [3]:
words = dataset["train"][0]["tokens"]
labels = dataset["train"][0]["ner_tags"]

for i in range(len(words)):
    print(f'{words[i]}\t{label_names[labels[i]]}')

EU	B-ORG
rejects	O
German	B-MISC
call	O
to	O
boycott	O
British	B-MISC
lamb	O
.	O


### Предобработка

На протяжении всего домашнего задания мы будем использовать _cased_ версию BERT, то есть токенизатор будет учитывать регистр слов. Для задачи NER регистр важен, так как имена и названия организаций или предметов искусства часто пишутся с большой буквы, и будет глупо прятать от модели такую информацию.

In [4]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

/Users/andreypetukhov/Documents/Машинное-обучение/.venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


При токенизации слова могут разделиться на несколько токенов (как слово `Fischler` из примера ниже), из-за чего появится несоответствие между числом токенов и меток. Это несоответствие нам придется устранить вручную.

In [5]:
example = dataset["train"][12]
words = example["tokens"]
tags = [label_names[id] for id in example["ner_tags"]]
tokenized_text = tokenizer(example["tokens"], is_split_into_words=True)

print('Слова: ', words)
print('Токены:', tokenized_text.tokens())
print('Метки:', tags)

Слова:  ['Only', 'France', 'and', 'Britain', 'backed', 'Fischler', "'s", 'proposal', '.']
Токены: ['[CLS]', 'Only', 'France', 'and', 'Britain', 'backed', 'Fi', '##sch', '##ler', "'", 's', 'proposal', '.', '[SEP]']
Метки: ['O', 'B-LOC', 'O', 'B-LOC', 'O', 'B-PER', 'O', 'O', 'O']


__Задание 1 (1 балл).__ Токенизируйте весь датасет и для каждого текста выравните токены с метками так, чтобы каждому токену соответствовала одна метка. При этом важно сохранить нотацию BIO. И не забудьте про специальные токены! Должно получиться что-то такое:

Теперь весь датасет:

In [11]:
def align_labels_with_tokens(labels: list, word_ids: list, label_names: list = label_names):
    aligned_labels = []
    prev_word_id = None
    for word_id in word_ids:
        if word_id is None:
            aligned_labels.append(-100)
        elif word_id != prev_word_id:
            aligned_labels.append(labels[word_id])
        else:
            aligned_label = labels[word_id]
            if label_names[aligned_label].startswith("B-"):
                aligned_label = label_names.index("I-" + label_names[aligned_label][2:])
            aligned_labels.append(aligned_label)
        prev_word_id = word_id
    return aligned_labels


def prepare_dataset(dataset: DatasetDict, tokenizer, label_names: list, max_length: int = 128):
    """
    Tokenizes and aligns labels for a given dataset.

    Args:
    - dataset (DatasetDict): The dataset containing tokens and ner_tags.
    - tokenizer: The tokenizer to use.
    - label_names (list): List of label names.
    - max_length (int): Maximum sequence length.

    Returns:
    - processed_dataset: The processed dataset ready for training.
    """
    def tokenize_and_align_labels(examples):
        tokenized_inputs: BatchEncoding = tokenizer(
            examples["tokens"],
            is_split_into_words=True,
            truncation=True,
            padding="max_length",
            max_length=max_length
        )
        
        all_labels = []
        all_tokens = []
        for i, labels in enumerate(examples["ner_tags"]):
            tokenizer_tokens = tokenized_inputs.tokens(batch_index=i)
            all_tokens.append(tokenizer_tokens)

            word_ids = tokenized_inputs.word_ids(batch_index=i)
            aligned_labels = align_labels_with_tokens(labels, word_ids)
            all_labels.append(aligned_labels)
        
        tokenized_inputs["labels"] = all_labels
        tokenized_inputs["tokenizer_tokens"] = all_tokens
        return tokenized_inputs

    processed_dataset = dataset.map(tokenize_and_align_labels, batched=True)
    processed_dataset = processed_dataset.remove_columns(["tokens", "ner_tags"])
    processed_dataset.set_format(type="torch", columns=["tokenizer_tokens", "input_ids", "attention_mask", "labels"])

    return processed_dataset


processed_dataset = prepare_dataset(dataset, tokenizer, label_names, max_length=60)
processed_dataset["train"][12]

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

{'input_ids': tensor([  101,  2809,  1699,  1105,  2855,  5534, 17355,  9022,  2879,   112,
           188,  5835,   119,   102,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'labels': tensor([-100,    0,    5,    0,    5,    0,    1,    2,    2,    0,    0,    0,
            0, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100

### Метрика

Для оценки качества NER обычно используют F1 меру с микро-усреднением. Мы загрузим ее из библиотеки `seqeval`. Функция `f1_score` принимает два 2d списка с правильными и предсказанными метками, записаными текстом, и возвращает для них значение F1. Вы можете использовать ее с параметрами по умолчанию.

In [68]:
from seqeval.metrics import f1_score

Особенность подсчета F1 для NER заключается в том, что в некоторых ситуациях неправильные ответы могут засчитываться как правильные. Например, если модель предсказала `['I-PER', 'I-PER']`, то мы можем догадаться, что на самом деле должно быть `['B-PER', 'I-PER']`, так как сущность не может начинаться с `I-`. Функция `f1_score` учитывает это и поэтому работает только с текстовыми представлениями меток.

### Модель

В качестве базовой модели мы возьмем `bert-base-cased`. Как вы понимаете, он не обучался на задачу NER. Поэтому прежде чем приступать к уменьшению размера BERT, его необходимо дообучить.

__Задание 2 (1 балл)__ Дообучите `bert-base-cased` на нашем датасете с помощью обычного fine-tuning. У вас должно получиться хотя бы 0.9 F1 на тестовой выборке. Заметьте, что чем выше качество большой модели, тем лучше будет работать дистиллированный ученик. Для обучения можно использовать `Trainer` из Hugging Face.

In [ ]:
model = AutoModelForTokenClassification.from_pretrained('bert-base-cased', num_labels=len(label_names))

print('Число параметров:', sum(p.numel() for p in model.parameters()))

In [44]:
# your code here

### Факторизация матрицы эмбеддингов

Можно заметить, что на данный момент матрица эмбеддингов занимает $V \cdot H = 28996 \cdot 768 = 22.268.928$ параметров. Это aж пятая часть от всей модели! Давайте попробуем что-то с этим сделать. В модели [ALBERT](https://arxiv.org/pdf/1909.11942.pdf) предлагается факторизовать матрицу эмбеддингов в произведение двух небольших матриц. Таким образом, параметры эмбеддингов будут содержать $V \cdot E + E \cdot H$ элементов, что гораздо меньше $V \cdot H$, если $H \gg E$. Авторы выбирают $E = 128$, однако ничего не мешает нам взять любое другое значение. Например, выбрав $H = 64$, мы уменьшим число параметров примерно на 20М.

__Задание 3 (1 балл).__ Напишите класс-обертку над слоем эмбеддингов, который реализует факторизацию на две матрицы, и дообучите факторизованную модель. Заметьте, обе матрицы можно инициализировать с помощью SVD разложения, чтобы начальное приближение было хорошим. Это сэкономит очень много времени на дообучении. С рангом разложения $H = 64$ у вас должно получиться F1 больше 0.87.

In [ ]:
import torch.nn as nn
from sklearn.decomposition import TruncatedSVD

class FactorEmbedding(nn.Module):
    def __init__(self, source_embed, factor_rank):
        super().__init__()
        self.source_embed = source_embed
        self.embedding_size = source_embed.embedding_dim
        self.vocab_size = source_embed.num_embeddings
        self.factor_rank = factor_rank

        self.embedding_matrix_left = nn.Embedding(self.vocab_size, self.factor_rank)
        self.embedding_matrix_right = nn.Parameter(torch.randn(self.factor_rank, self.embedding_size))

        self.init_svd()

    def init_svd(self):
        source_weight = self.source_embed.weight.data.cpu().numpy()
        svd = TruncatedSVD(n_components=self.factor_rank)
        U, S, Vt = svd.fit_transform(source_weight), svd.singular_values_, svd.components_
        self.embedding_matrix_left.weight.data.copy_(torch.from_numpy(U))
        self.embedding_matrix_right.data = torch.tensor(np.diag(S).dot(Vt), dtype=torch.float32)

    def forward(self, x):
        left = self.embedding_matrix_left(x)
        return nn.functional.linear(left, self.embedding_matrix_right)

### Дистилляция знаний

Дистилляция знаний – это парадигма обучения, в которой знания модели-учителя дистиллируются в модель-ученика. Учеником может быть произвольная модель меньшего размера, решающая ту же задачу, однако обычно ученик имеет ту же архитектуру, что и учитель. При дистилляции используются два функционала ошибки:

1. Стандартная кросс-энтропия.
1. Функция, задающая расстояние между распределениями предсказаний учителя и ученика. Чаще всего используют KL-дивергенцию.

Для того, чтобы распределение предсказаний учителя не было вырожденным, к softmax добавляют температуру больше 1, например, 2 или 5.   
__Важно:__ при делении логитов на температуру значения градиентов уменьшаются в $\tau^2$ раз (проверьте это!). Поэтому для возвращения их в изначальный масштаб ошибку надо домножить на $\tau^2$. Подробнее об этом можно почитать в разделе 2.1 [оригинальной статьи](https://arxiv.org/pdf/1503.02531).

<img src="https://intellabs.github.io/distiller/imgs/knowledge_distillation.png" width="1000">

__Задание 4 (3 балла).__ Реализуйте метод дистилляции знаний, изображенный на картинке. Для подсчета ошибки между предсказаниями ученика и учителя используйте KL-дивергенцию [`nn.KLDivLoss(reduction="batchmean")`](https://pytorch.org/docs/stable/generated/torch.nn.KLDivLoss.html) (обратите внимание на вормат ее входов). Для получения итоговой ошибки суммируйте мягкую ошибку с жесткой.   
В качестве учителя используйте дообученный BERT из задания 2. В качестве ученика возьмите необученную модель с размером __не больше 20M__ параметров. Вы можете использовать факторизацию матрицы эмбеддингов для уменьшения числа параметров. Если вы все сделали правильно, то на тестовой выборке вы должны получить значение F1 не меньше 0.7. Вам должно хватить примерно 20к итераций обучения для этого. Если у вас что-то не получается, то можно ориентироваться на статью про [DistilBERT](https://arxiv.org/abs/1910.01108) и на [эту статью](https://www.researchgate.net/publication/375758425_Knowledge_Distillation_Scheme_for_Named_Entity_Recognition_Model_Based_on_BERT).

__Важно:__
* Не забывайте добавлять _warmup_ при обучении ученика.
* Не забывайте переводить учителя в режим _eval_.

In [ ]:
# your code here

# Задания на выбор

Как вы понимаете, есть еще довольно много разных способов уменьшить обученную модель. В этой секции вам предлагается реализовать разные техники на выбор. За каждую из них можно получить разное количество балов в зависимости от сложности. Успешность реализации будет оцениваться как по коду, так и по качеству на тестовой выборке. Все баллы за это дз, которые вы наберете сверх 10, будут считаться бонусными.   
В задании 4 вы обучали модель с ограничением числа параметров в 20М. При реализации техник из этой секции придерживайтесь такого же ограничение. Это позволит честно сравнивать методы между собой и делать правильные выводы. Напишите в отчете обо всем, что вы попробовали.

* __Шеринг весов (2 балла).__ В модификации BERT [ALBERT](https://arxiv.org/pdf/1909.11942.pdf) помимо факторизации эмбеддингов предлагается шерить веса между слоями. То есть разные слои используют одни и те же веса. Такая техника эвивалентна применению одного и того же слоя несколько раз. Она позволяет в несколько раз уменьшить число параметров и не сильно потерять в качестве.
* __Факторизация промежуточных слоев (2 балла).__ Если можно факторизовать матрицу эмбеддингов, то и все остальное тоже можно. Для факторизации слоев существует много разных подходов и выбрать какой-то один сложно. Вы можете вдохновляться [этим списком](https://lechnowak.com/posts/neural-network-low-rank-factorization-techniques/), найти в интернете что-то другое или придумать метод самостоятельно. В любом случае в отчете обоснуйте, почему вы решили сделать так как сделали.
* __Приближение промежуточных слоев (2 балла).__ Мы обсуждали, что помимо приближения выходов модели ученика к выходам модели учителя, можно приближать выходы промежуточных слоев. В [этой работе](https://www.researchgate.net/publication/375758425_Knowledge_Distillation_Scheme_for_Named_Entity_Recognition_Model_Based_on_BERT) подробно написано, как это можно сделать.
* __Прунинг (4 балла).__ В методе [SparseGPT](https://arxiv.org/abs/2301.00774) предлагается подход, удаляющий веса модели один раз после обучения. При этом оказывается возможным удалить до половины всех весов без потери в качестве. Математика, стоящаяя за техникой, довольно сложная, однако общий подход простой – будем удалять веса в каждом слое по отдельности, при удалении части весов слоя, остальные веса будут перенастраиваться так, чтобы общий выход слоя не изменился.
* __Удаление голов (6 баллов).__ В данный момент мы используем все головы внимания, но ряд исследований показывает, что большинство из них можно выбросить без потери качества. В этой [статье](https://arxiv.org/pdf/1905.09418.pdf) предлагается подход, который добавляет гейты к механизму внимания, которые регулируют, какие головы участвуют в слое, а какие – нет. В процессе обучения гейты настраиваются так, чтобы большинство голов не использовалась. В конце обучения неиспользуемые головы можно удалить. За это задание дается много баллов, потому что в методе довольно сложная математика и подход плохо заводится. Если вы решитесь потратить на него свои силы, то в случае неудачи мы дадим промежуточные баллы, опираясь на отчет.   
__Совет:__ во время обучения внимательно следите за поведением гейтов. Если вы все сделали правильно, то они должны зануляться. Однако зануляются они не всегда сразу, им надо дать время и обучать модель подольше.

In [139]:
from transformers import BertConfig, BertForTokenClassification
config = BertConfig.from_pretrained('bert-base-cased')
device = torch.device("cpu")

In [140]:
config.num_hidden_layers = 4
config.hidden_size = 192
config.num_labels = len(label_names)
model_big = BertForTokenClassification(config).to(device)

In [129]:
class SharedAttentionBert(BertForTokenClassification):
    def __init__(self, config):
        super().__init__(config)
        self.shared_attention = self.bert.encoder.layer[0].attention
        # БОО
        for i in range(1, len(self.bert.encoder.layer)):
            self.bert.encoder.layer[i].attention = self.shared_attention

        self.param_savings = self.calculate_param_savings()

    def calculate_param_savings(self):
        attention = self.shared_attention
        hidden_size = attention.self.query.weight.size(0)
        num_attention_heads = attention.self.num_attention_heads
        param_per_attention_layer = (
            (hidden_size * hidden_size) * 3 + hidden_size * hidden_size
        ) * num_attention_heads

        num_layers = len(self.bert.encoder.layer)
        param_savings = (num_layers - 1) * param_per_attention_layer
        
        return param_savings

model_small = SharedAttentionBert(config).to(device)

print(f"Экономия параметров: {model_small.param_savings} параметров.")

Экономия параметров: 5308416 параметров.


In [131]:
sum(p.numel() for p in model_big.parameters())

10995657

In [104]:
class FactorizedLinear(nn.Module):
    def __init__(self, in_features, out_features, rank=None):
        super(FactorizedLinear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features

        self.rank = rank if rank else min(in_features, out_features)
        self.U = nn.Parameter(torch.randn(in_features, self.rank))
        self.V = nn.Parameter(torch.randn(self.rank, out_features))
        self.W = torch.matmul(self.U, self.V)

    def forward(self, x):
        return nn.functional.linear(x, self.W)

In [132]:
class FactorizedBertIntermediate(nn.Module):
    def __init__(self, d_model: int, d_ff: int, k: int):
        super(FactorizedBertIntermediate, self).__init__()
        self.d_model = d_model
        self.d_ff = d_ff
        self.k = k
        self.U = nn.Parameter(torch.randn(d_model, k))
        self.S = nn.Parameter(torch.randn(k, k))
        self.V = nn.Parameter(torch.randn(k, d_ff))

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        intermediate_output = torch.matmul(torch.matmul(x, self.U), self.S)
        return torch.matmul(intermediate_output, self.V)

class FactorizedBertOutput(nn.Module):
    def __init__(self, d_ff: int, d_model: int, k: int):
        super(FactorizedBertOutput, self).__init__()
        self.d_model = d_model
        self.d_ff = d_ff
        self.k = k
        self.U = nn.Parameter(torch.randn(d_ff, k))
        self.S = nn.Parameter(torch.randn(k, k))
        self.V = nn.Parameter(torch.randn(k, d_model))

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        output = torch.matmul(torch.matmul(x, self.U), self.S)
        return torch.matmul(output, self.V)


def replace_with_factorized_layers(model: nn.Module, k: int):
    for layer in model.bert.encoder.layer:
        intermediate_dense = layer.intermediate.dense
        d_model = intermediate_dense.in_features
        d_ff = intermediate_dense.out_features
        layer.intermediate.dense = FactorizedBertIntermediate(d_model, d_ff, k)

        output_dense = layer.output.dense
        d_ff = output_dense.in_features
        d_model = output_dense.out_features
        layer.output.dense = FactorizedBertOutput(d_ff, d_model, k)
    
    print("Замена слоёв завершена.")

k = 64

replace_with_factorized_layers(model_big, k)

print(model_big.bert.encoder.layer[0].intermediate.dense)
print(model_big.bert.encoder.layer[0].output.dense) 

Замена слоёв завершена.
FactorizedBertIntermediate()
FactorizedBertOutput()


In [133]:
sum(p.numel() for p in model_big.parameters())

7967945

## Pruning

In [134]:
import math
import time

import torch
import torch.nn as nn
import transformers


DEBUG = False 

torch.backends.cuda.matmul.allow_tf32 = False
torch.backends.cudnn.allow_tf32 = False

class SparseGPT:

    def __init__(self, layer):
        self.layer = layer
        self.dev = self.layer.weight.device
        W = layer.weight.data.clone()
        if isinstance(self.layer, nn.Conv2d):
            W = W.flatten(1)
        if isinstance(self.layer, transformers.Conv1D):
            W = W.t()
        self.rows = W.shape[0]
        self.columns = W.shape[1]
        self.H = torch.zeros((self.columns, self.columns), device=self.dev)
        self.nsamples = 0

    def add_batch(self, inp, out, blocksize=1024):
        if DEBUG:
            self.inp1 = inp
            self.out1 = out
        if len(inp.shape) == 2:
            inp = inp.unsqueeze(0)
        tmp = inp.shape[0]
        if isinstance(self.layer, nn.Linear) or isinstance(self.layer, transformers.Conv1D):
            if len(inp.shape) == 3:
                inp = inp.reshape((-1, inp.shape[-1]))
            inp = inp.t()
        self.H *= self.nsamples / (self.nsamples + tmp)
        self.nsamples += tmp
        inp = math.sqrt(2 / self.nsamples) * inp.float()
        self.H += inp.matmul(inp.t())

    def fasterprune(
        self, sparsity, prunen=0, prunem=0, blocksize=128, percdamp=.01
    ):
        W = self.layer.weight.data.clone()
        if isinstance(self.layer, nn.Conv2d):
            W = W.flatten(1)
        if isinstance(self.layer, transformers.Conv1D):
            W = W.t()
        W = W.float()

        tick = time.time()

        H = self.H
        del self.H
        dead = torch.diag(H) == 0
        H[dead, dead] = 1
        W[:, dead] = 0

        Losses = torch.zeros(self.rows, device=self.dev)

        damp = percdamp * torch.mean(torch.diag(H))
        diag = torch.arange(self.columns, device=self.dev)
        H[diag, diag] += damp
        H = torch.linalg.cholesky(H)
        H = torch.cholesky_inverse(H)
        H = torch.linalg.cholesky(H, upper=True)
        Hinv = H

        mask = None

        for i1 in range(0, self.columns, blocksize):
            i2 = min(i1 + blocksize, self.columns)
            count = i2 - i1

            W1 = W[:, i1:i2].clone()
            Q1 = torch.zeros_like(W1)
            Err1 = torch.zeros_like(W1)
            Losses1 = torch.zeros_like(W1)
            Hinv1 = Hinv[i1:i2, i1:i2]

            if prunen == 0: 
                if mask is not None:
                    mask1 = mask[:, i1:i2]
                else:
                    tmp = W1 ** 2 / (torch.diag(Hinv1).reshape((1, -1))) ** 2
                    thresh = torch.sort(tmp.flatten())[0][int(tmp.numel() * sparsity)]
                    mask1 = tmp <= thresh
            else:
                mask1 = torch.zeros_like(W1) == 1

            for i in range(count):
                w = W1[:, i]
                d = Hinv1[i, i]

                if prunen != 0 and i % prunem == 0:
                    tmp = W1[:, i:(i + prunem)] ** 2 / (torch.diag(Hinv1)[i:(i + prunem)].reshape((1, -1))) ** 2
                    mask1.scatter_(1, i + torch.topk(tmp, prunen, dim=1, largest=False)[1], True)

                q = w.clone()
                q[mask1[:, i]] = 0

                Q1[:, i] = q
                Losses1[:, i] = (w - q) ** 2 / d ** 2

                err1 = (w - q) / d
                W1[:, i:] -= err1.unsqueeze(1).matmul(Hinv1[i, i:].unsqueeze(0))
                Err1[:, i] = err1

            W[:, i1:i2] = Q1
            Losses += torch.sum(Losses1, 1) / 2

            W[:, i2:] -= Err1.matmul(Hinv[i1:i2, i2:])

            if DEBUG:
                self.layer.weight.data[:, :i2] = W[:, :i2]
                self.layer.weight.data[:, i2:] = W[:, i2:]
                print(torch.sum((self.layer(self.inp1) - self.out1) ** 2))
                print(torch.sum(Losses))

        torch.cuda.synchronize()
        print('time %.2f' % (time.time() - tick))
        print('error', torch.sum(Losses).item())

        if isinstance(self.layer, transformers.Conv1D):
            W = W.t()
        self.layer.weight.data = W.reshape(self.layer.weight.shape).to(self.layer.weight.data.dtype)
        if DEBUG:
            print(torch.sum((self.layer(self.inp1) - self.out1) ** 2))

    def free(self):
        if DEBUG:
            self.inp1 = None
            self.out1 = None
        self.H = None
        torch.cuda.empty_cache()

In [141]:
print(sum(p.numel() for p in model_big.parameters()))

10995657


In [142]:
import copy

# Загружаем модель
model = copy.deepcopy(model_big)

# SparseGPT класс (должен быть определен заранее)
# Вставь свой код класса SparseGPT сюда

# Переменные для хранения входов/выходов
captured_inputs = {}
captured_outputs = {}

# Hook-функция
def hook_fn(layer_name):
    def hook(module, inputs, outputs):
        global captured_inputs, captured_outputs
        captured_inputs[layer_name] = inputs[0].detach()
        captured_outputs[layer_name] = outputs.detach()
    return hook

# Регистрация хуков для всех линейных слоев
hooks = []
for name, module in model.named_modules():
    if isinstance(module, nn.Linear):
        hooks.append((name, module.register_forward_hook(hook_fn(name))))

# Пример входных данных
input_ids = torch.randint(0, 100, (8, 128)).to(device)  # Входные данные
attention_mask = torch.ones_like(input_ids).to(device)

# Прогон через модель для захвата входов/выходов
model(input_ids, attention_mask=attention_mask)

# Удаляем хуки после захвата данных
for _, hook in hooks:
    hook.remove()

# Применение SparseGPT ко всем линейным слоям
sparsity = 0.4  # 40% разреженности
for name, module in model.named_modules():
    if isinstance(module, nn.Linear):
        print(f"Pruning layer: {name}")
        
        # Создаем SparseGPT объект
        sparse_gpt = SparseGPT(module)
        
        # Добавляем данные слоя в SparseGPT
        inp = captured_inputs[name]
        out = captured_outputs[name]
        sparse_gpt.add_batch(inp, out, blocksize=128)
        
        # Выполняем прунинг
        sparse_gpt.fasterprune(sparsity=sparsity, blocksize=128, percdamp=0.01)
        
        # Освобождаем ресурсы
        sparse_gpt.free()

print("Pruning complete!")


Pruning layer: bert.encoder.layer.0.attention.self.query


AssertionError: Torch not compiled with CUDA enabled